In [1]:
import urllib
import urllib.request
import http
import http.cookiejar
import os

from bs4 import BeautifulSoup
import lxml

import MySQLdb

In [2]:
html_url = "https://www.meitulu.com/item/{}.html"
pic_url = "https://mtl.gzhuibei.com/images/img/{}/{}.jpg"

In [13]:
opener = urllib.request.build_opener()
#添加headers
headers = [('Referer',html_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',html_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers

In [3]:
## /html/body/div[2]/div[2]/p[2]
def get_gallery(gallery_num):
    try:
        print(html_url.format(gallery_num))
        dict_gallery_properties = {'gallery_num':gallery_num,'gallery_model':"", 'gallery_vol':"", 'gallery_pic_num':"", 'gallery_producer':"", 
                           'gallery_resolution':"", 'gallery_title':"", 'gallery_remark':[], 'gallery_date':"", 'gallery_raw':""}
        print(dict_gallery_properties)
        request = opener.open(html_url.format(gallery_num), timeout=60)
        request_read = request.read()
        #print(request_read.decode(encoding='UTF8',errors='ignore'))
        html_soup = BeautifulSoup(request_read)
        gallery_title = html_soup.title.text
        dict_gallery_properties["gallery_title"] = gallery_title[:-4].replace("/","")
        
        gallery_properties = html_soup.html.body.find_all(class_="c_l")[0].find_all("p")
        dict_gallery_properties['gallery_raw'] = str(gallery_properties)

        #dict_gallery_properties = 
        get_gallery_properties(gallery_properties, dict_gallery_properties)
        print(dict_gallery_properties)
        try:
            add_database_gallery(dict_gallery_properties)
            get_download(dict_gallery_properties)
        except Exception as e:
            print(e)
            update_crawl_end_time(dict_gallery_properties, str(e))
    except Exception as e:
        print(str(e))
        add_database_gallery(dict_gallery_properties)
        update_crawl_end_time(dict_gallery_properties, str(e))

In [5]:
def get_gallery_properties(gallery_properties, dict_gallery_properties):
    """分析图册属性"""
    for i in gallery_properties:
        print(i.text)
        if i.text.split("：")[0] == "发行机构":
            dict_gallery_properties["gallery_producer"] = i.text.split("：")[1].strip()
        elif i.text.split("：")[0] == "期刊编号":
            dict_gallery_properties["gallery_vol"] = i.text.split("：")[1].strip()
        elif i.text.split("：")[0] == "图片数量":
            dict_gallery_properties["gallery_pic_num"] = int(i.text.split("：")[1].replace("张","").strip())
        elif i.text.split("：")[0] == "分 辨 率":
            dict_gallery_properties["gallery_resolution"] = i.text.split("：")[1].strip()
        elif i.text.split("：")[0] == "模特姓名":
            dict_gallery_properties["gallery_model"] = i.text.split("：")[1].strip()
        elif i.text.split("：")[0] == "发行时间":
            dict_gallery_properties["gallery_date"] = i.text.split("：")[1].strip()
        else:
            dict_gallery_properties["gallery_remark"].append(i.text)
    #print(dict_gallery_properties)
    #return dict_gallery_properties

In [11]:
def get_download(dict_gallery_properties):
    """下载图册"""
    
    # 生成下载脚本
    with open('{}_{}.sh'.format(dict_gallery_properties['gallery_num'],dict_gallery_properties["gallery_model"]), "w") as f:
        f.write('mkdir "{}_{}" \n'.format(dict_gallery_properties['gallery_num'],dict_gallery_properties["gallery_title"]))
        f.write('cd "{}_{}" \n'.format(dict_gallery_properties['gallery_num'],dict_gallery_properties["gallery_title"]))
        for i in range(dict_gallery_properties["gallery_pic_num"]):
            f.write('wget {} -O "{}_{}_{}.jpg" -a ../log/{}_{}.log \n'.
                    format(pic_url.format(dict_gallery_properties['gallery_num'],i+1),dict_gallery_properties['gallery_num'],
                           dict_gallery_properties["gallery_model"],i+1,dict_gallery_properties['gallery_num'],
                           dict_gallery_properties["gallery_model"]))
            #print('wget {} -O "{}_{}_{}.jpg"'.format(pic_url.format(gallery_num,i+1),gallery_num,dict_gallery_properties["gallery_model"],i+1))
    
    update_crawl_start_time(dict_gallery_properties)        
    #os.system('sh "{}_{}.sh"'.format(gallery_num,dict_gallery_properties["gallery_model"]))
    update_crawl_end_time(dict_gallery_properties, '1')
    

In [7]:
def add_database_gallery(dict_gallery_properties):
    """将图库信息插入数据库"""
    spider_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8' )

    spider_cursor = spider_db.cursor()

    sql_stmt = """INSERT into t_meitulu_spider(gallery_num, gallery_title, gallery_producer, gallery_vol, 
    gallery_pic_num, gallery_resolution, gallery_model, gallery_date, gallery_remark, gallery_raw,get_date)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,SYSDATE())"""

    spider_cursor.execute(sql_stmt, (dict_gallery_properties['gallery_num'], dict_gallery_properties['gallery_title'], 
                                     dict_gallery_properties['gallery_producer'],dict_gallery_properties['gallery_vol'],
                                     dict_gallery_properties['gallery_pic_num'], dict_gallery_properties['gallery_resolution'], 
                                     dict_gallery_properties['gallery_model'], dict_gallery_properties['gallery_date'], 
                                     str(dict_gallery_properties['gallery_remark']), dict_gallery_properties['gallery_raw']))

    spider_db.commit()
    spider_cursor.close()
    spider_db.close()

In [8]:
def update_crawl_start_time(dict_gallery_properties):
    """爬虫抓取时间入数据库"""
    spider_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8' )

    spider_cursor = spider_db.cursor()

    sql_stmt = """UPDATE t_meitulu_spider a set a.crawl_start_date = SYSDATE()
    WHERE a.gallery_num = %s and a.gallery_model = %s"""

    spider_cursor.execute(sql_stmt, (dict_gallery_properties['gallery_num'], dict_gallery_properties['gallery_model']))

    spider_db.commit()
    spider_cursor.close()
    spider_db.close()

In [9]:
def update_crawl_end_time(dict_gallery_properties, spider_status):
    """爬虫抓取状态入数据库"""
    spider_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8' )

    spider_cursor = spider_db.cursor()

    sql_stmt = """UPDATE t_meitulu_spider a set a.crawl_end_date = SYSDATE(), a.crawl_status = %s 
    WHERE a.gallery_num = %s and a.gallery_model = %s"""

    spider_cursor.execute(sql_stmt, (spider_status, dict_gallery_properties['gallery_num'], dict_gallery_properties['gallery_model']))

    spider_db.commit()
    spider_cursor.close()
    spider_db.close()

In [10]:
get_gallery(1)

https://www.meitulu.com/item/1.html
{'gallery_num': 1, 'gallery_model': '', 'gallery_vol': '', 'gallery_pic_num': '', 'gallery_producer': '', 'gallery_resolution': '', 'gallery_title': '', 'gallery_remark': [], 'gallery_date': '', 'gallery_raw': ''}
1
HTTP Error 404: Not Found
2
3
4


In [7]:
html_url = "https://www.meitulu.com/item/{}.html".replace("/","").replace("\\","")

In [5]:
html_url

'https:www.meitulu.comitem{}.html'

In [20]:
request = opener.open(html_url.format(3), timeout=1)
request_read = request.read()
print(request_read.decode(encoding='UTF8',errors='ignore'))

URLError: <urlopen error timed out>

In [ ]:
add_database_gallery(dict_gallery_properties)

In [ ]:
dict_gallery_properties.keys()

In [ ]:
dict_gallery_properties["gallery_remark"].append('i.text123')

In [ ]:
dict_gallery_properties

In [ ]:
import MySQLdb

In [ ]:
dict_gallery_properties['gallery_number']=7

In [ ]:
(dict_gallery_properties['gallery_number'], dict_gallery_properties['gallery_title'], dict_gallery_properties['gallery_producer'],
 dict_gallery_properties['gallery_pic_num'], dict_gallery_properties['gallery_resolution'], dict_gallery_properties['gallery_model'], 
 dict_gallery_properties['gallery_date'], str(dict_gallery_properties['gallery_remark']), dict_gallery_properties['gallery_raw'])

In [2]:
spider_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8' )
spider_cursor = spider_db.cursor()

In [4]:
spider_cursor.close()
spider_db.close()

OperationalError: (2006, '')

In [ ]:
a.append('123')

In [ ]:
range(1,10)

In [ ]:
class Gallery():
    """图册的类"""
    def __init__(self, gallery_number):
        """通过图册编号进行初始化"""
        self.gallery_number = gallery_number
        
    def get_html(self):
        

In [ ]:
url1 = 'http://www.itokoo.com/'
url2 = 'http://www.itokoo.com/login.php'
'''headers = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",\
    "User-Agent": "Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36",\
    "Connection": "Keep-Alive",\
    "Cache-Control": "no-cache"}'''
opener = urllib.request.build_opener()
#添加headers
headers = [('Referer',url1),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',url1),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers

#添加cookie
cookie_filename = 'cookie_itoko.txt'
#cookies = http.cookiejar.CookieJar()
cookies = http.cookiejar.MozillaCookieJar(cookie_filename)
cookies.load(ignore_discard=True, ignore_expires=True)
cookie_handler = urllib.request.HTTPCookieProcessor(cookies)
opener.add_handler(cookie_handler)
#添加代理
proxy_handler = urllib.request.ProxyHandler({'https':'221.178.146.113:61399','http':'221.178.146.113:61399'})
#opener.add_handler(proxy_handler)

#添加PostData
data = {"jumpurl":"http%3A%2F%2Fwww.itokoo.com%2Findex.php","step":"2",\
        "ajax":"1","pwuser":"Flashield","pwpwd":"Zb122525","lgt":"0"}
post_data = urllib.parse.urlencode(data).encode(encoding='UTF8')
print(post_data)

In [ ]:
#request = opener.open(url1)
#request_read = request.read()
#print(request_read.decode(encoding='UTF8',errors='ignore'))


In [ ]:
request = opener.open(url2,post_data)
print(request.read().decode(encoding='UTF8',errors='ignore'))

In [ ]:
#登录成功后，保存cookies
cookies.save(ignore_discard=True, ignore_expires=True)

In [ ]:
url3='http://www.itokoo.com/jobcenter.php?action=apply'
headers = [('Referer',url3),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',url3),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers
data3 = {"id":"5","step":"2"}
post_data3 = urllib.parse.urlencode(data3).encode(encoding='UTF8')
request = opener.open(url3,post_data3)
print(request.read().decode(encoding='GBK',errors='ignore'))

In [ ]:
url4='http://www.itokoo.com/jobcenter.php?action=gain'
data4 = {"id":"5","step":"2"}
post_data4 = urllib.parse.urlencode(data4).encode(encoding='UTF8')
request = opener.open(url4,post_data4)
print(request.read().decode(encoding='GBK',errors='ignore'))
#print(request_read.decode(encoding='UTF8',errors='ignore'))

In [ ]:
url5='http://www.itokoo.com/jobcenter.php?action=punch'
headers5 = [('Referer','http://www.itokoo.com/u.php'),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin','http://www.itokoo.com/u.php'),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers5
data5 = {"step":"2"}
post_data5 = urllib.parse.urlencode(data5).encode(encoding='UTF8')
request = opener.open(url4,post_data4)
print(request.read().decode(encoding='GBK',errors='ignore'))